&#x1f12f; Javier Bejar - APA/GEI/FIB/UPC (2022)

In [1]:
# Uncomment to upgrade packages
# !pip install pandas --user --upgrade --quiet
# !pip install numpy --user --upgrade --quiet
# !pip install scipy --user --upgrade --quiet
# !pip install seaborn --user --upgrade --quiet
# !pip install matplotlib --user --upgrade --quiet
# !pip install scikit-learn --user --upgrade 
# !pip install scikit-optimize --user --quiet
# !pip install eli5 --user --quiet
!pip install apafib --upgrade --user --quiet
!pip install wordcloud --upgrade --user --quiet


In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display, HTML
show_html = lambda html: display(HTML(html))

from time import time
from datetime import timedelta

init_time = time()

# APA - Laboratorio - Sesión 6
## Support Vector Machines

In [ ]:
import pandas as pd
from pandas import read_csv

import numpy as np
from numpy.random import choice
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn import set_config

from sklearn.metrics import  ConfusionMatrixDisplay,\
                    classification_report,  RocCurveDisplay, PrecisionRecallDisplay,\
                    accuracy_score, f1_score, precision_score, recall_score

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn.svm import LinearSVC, SVC
from sklearn.svm import LinearSVR, SVR

from sklearn.metrics import mean_squared_error, make_scorer, mean_absolute_error
from yellowbrick.classifier.rocauc import roc_auc

from sklearn.inspection import permutation_importance
from sklearn.model_selection import GridSearchCV,train_test_split, cross_val_score, TimeSeriesSplit
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from apafib import load_arxiv, load_energy

from skopt import BayesSearchCV

import warnings

set_config(display='text')
warnings.filterwarnings('ignore')
plt.rcParams.update({'font.size': 16})
# sns.set()
pd.set_option('display.precision', 3)

In [ ]:
def save_results(clf, X_test, y_test, nclf, df):
    df.loc[nclf,'test acc'] = accuracy_score(y_test, clf.predict(X_test))
    df.loc[nclf,'precision score (W)'] = precision_score(y_test, clf.predict(X_test), average='weighted')
    df.loc[nclf,'recall score (W)'] = recall_score(y_test, clf.predict(X_test), average='weighted')
    df.loc[nclf,'f1 score (W)'] = f1_score(y_test, clf.predict(X_test), average='weighted')
    return df

results_df = pd.DataFrame()

niter = 15
cv = 5

# Sección 1: Arxiv abstracts (Clasificación)

No todos los conjuntos de datos que se usan para generar modelos corresponden a datos tabulares, en algunos problemas debemos transformar datos no estructurados de alguna manera para poder usarlos.

Este es el caso del texto. Antes de que se pueda usar debemos realizar un preproceso que nos de una matriz de datos sobre la que podamos aplicar un modelo.

En este caso particular el proceso habitual sigue los siguientes pasos (los que hayáis o estéis haciendo CAIM ya los conocéreis)

1. Tokenización: Dividir el texto en palabras (usando espacios en blanco o expresiones regulares)
2. Normalización: Transformar los token a un formato único para que la misma palabra (minúsculas, eliminación de acentos, caracteres no ASCII...)
3. Eliminación de stop words: Quitar palabras que no tienen significiado por si mismas (preposiciones, adverbios, artículos). Claramente depende del idioma
4. Lematización: Transformación de las palabras a su raíz. Esto también depende del idioma
5. Reducción del vocabulario a un rango de frecuencias (ni las más frecuentes, ni las muy poco frecuentes)
6. Transformación de cada texto a un vector de características: 
   - Binario (la palabra esta o no)
   - Conteo (Cuantas veces aparece la palabra)  
   - Importancia de la palabra respecto al conjunto de documentos, por ejemplo TFIDF
        TF = term frequency, cuantas veces aparece la palabra en el documento
        IDF = inverse document frequency, en cuantos documentos aparece la palabra

Esta representación del texto se conoce como **Bag of words**, representa cada documento por una lista de palabras perteneciente a un vocabulario y un atributo calculado para cada uno de ellos.

Una vez tenemos los datos como una matriz podemos aplicar cualquier modelo.

En este caso trabajaremos con un conjunto de datos que corresponde a resúmenes de artículos científicos extraidos de Arxiv (https://arxiv.org/). Podemos obtener los textos mediante la función de _apafib_ `load_arxiv`. Esto nos retornara dos listas, una con las etiquetas de los documentos y otra con los documentos.

En este conjunto de datos tenemos articulos de cuatro categorías: astro-ph, cs, math, physics. Hay 1000 ejemplos de cada categoría.

Comenzaremos cargando los datos

In [ ]:
text, labels = load_arxiv()

Tenemos 4000 ejemplos en total

In [ ]:
len(text), len(labels)

Este es el primer ejemplo

In [ ]:
text[0], labels[0]

Antes de procesar el texto lo dividiremos en conjunto de entrenamiento y test, también transformaremos la etiquetas textuales a números

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(text, labels, test_size=0.3, random_state=42, stratify=labels)

In [ ]:
lenc = LabelEncoder()
y_train_l = lenc.fit_transform(y_train)
y_test_l = lenc.transform(y_test)
cls = lenc.inverse_transform(np.unique(y_train_l))

In [ ]:
X_train[0]

### Transformación a matriz de datos

Consideraremos diferentes formas de transformar el texto a una matriz de datos, para este fin scikit-learn tiene dos funciones especiales para ello:

-  `CountVectorizer` que permite obtener el conteo de palabras, pero también la representación binaria
-  `TfidfVectorizer` que calcula el TFIDF de las palabras


Empezaremos por el mas sencillo, una representación binaria. 

Tendremos que elegir todo un conjunto de parámetros sobre como se preprocesa y genera el vocabulario del texto, algunos los dejaremos por defecto, pero serían parte de los hiperparámetros que tendremos que explorar para encontrar el mejor modelo. Podéis verlos en (https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer)

Modificaremos los siguientes:
 - `max_features` el número de palabras en el vocabulario, por defecto se escogen las más frecuentes que podría no ser la mejor opción, se puede controlar el rango de frecuencias que se escoge con otros paámetros, probaremos varias opciones
 - `stop_words` que pondremos a `english`

El vectorizador nos retorna una matriz esparsa cuando tiene sentido, esto permitirá reducir el coste en memoria, pero no todos los modelos pueden trabajar con esta representación.

In [ ]:
voc_size = 1000

cvec = CountVectorizer(stop_words='english', max_features=voc_size, binary=True)

X_train_v = cvec.fit_transform(X_train)
X_test_v = cvec.transform(X_test)
X_train_v.shape

Podemos visualizar el vocabulario que tenemos

In [ ]:
from wordcloud import WordCloud

wordcloud = WordCloud(background_color='white').generate_from_frequencies(cvec.vocabulary_)
plt.figure(figsize=(12,8))
plt.imshow(wordcloud, interpolation='bilinear');
plt.axis('off');

### Visualización

Claramente no podemos hacer un análisis exploratorio de las variables o una visualización detallada, asi que utilizaremos reducción de dimensionalidad para observar la relación entre los atributos y las clases.

Comenzaremos con PCA aunque claramente los atributos no se distribuyen de manera gausiana.

In [ ]:
pca = PCA()
tdata = pca.fit_transform(X_train_v.todense())
dfdata = pd.concat([pd.DataFrame(tdata[:,:2]),pd.DataFrame({'label':y_train})],axis=1)
fig = plt.figure(figsize=(8,8))
sns.scatterplot(x=0, y=1, hue='label', data=dfdata, palette='tab10');

Claramente tampoco podemos esperar que unos pocos componentes expliquen la variancia de los datos.

In [ ]:
fig = plt.figure(figsize=(8,6));
plt.plot(range(1,len(pca.explained_variance_ratio_ )+1),pca.explained_variance_ratio_ ,alpha=0.8,marker='.',label="Variancia Explicada");
y_label = plt.ylabel('Variancia explicada');
x_label = plt.xlabel('Componentes');
plt.plot(range(1,len(pca.explained_variance_ratio_ )+1),
         np.cumsum(pca.explained_variance_ratio_),
         c='red',marker='.',
         label="Variancia explicada acumulativa");
plt.legend();
plt.title('Porcentaje de variancia explicada por componente');

Podemos ver que hay cierta separabilidad entre las clases, también la proyección nos da una idea de las relaciones entre ellas.

Podemos usar también t-SNE dado que trabaja directamente con distancias. Lo inicializaremos con PCA.

In [ ]:
tsne = TSNE(init='pca')
tdata = tsne.fit_transform(X_train_v.todense())
dfdata = pd.concat([pd.DataFrame(tdata[:,:2]),pd.DataFrame({'label':y_train})],axis=1);
fig = plt.figure(figsize=(8,8));
sns.scatterplot(x=0, y=1, hue='label', data=dfdata, palette='tab10');

### Support Vector Machines

La idea de una SVN es encontrar el separador óptimo entre las clases que corresponderá con el hiperplano que maximice el margen entre ellas. Vimos que esto se puede obtener resolviendo un problema de programación cuadrática.

Si las clases no son linealemente separables (como probablemente pasa en nuestro caso) podemos poner un límite al error que comete el clasificador para encontrar el separador. También podemos utilizar diferentes kernels que aumentan la dimensionalidad del espacio para obtener más fácilmente el separador.

En este caso probaremos la SVM lineal, la SVM con kernel polinómico y la SVM con kernel RBF. 

Desde el punto de vista de la interpretabilidad sería mejor la SVM lineal, pero interpretar los pesos en este problema puede ser complicado, veremos más adelante como se pueden obtener explicaciones de la clasificación de ejemplos en este caso.

#### Kernel lineal

Empezaremos con la SVM lineal.

Antes de usarla lo usual es normalizar los datos para usar la SVM. En este caso no lo necesitamos ya que los datos son binarios, pero podría ayudar a la convergencia en otros casos.


In [ ]:
param = {'C':10**np.linspace(-3,3,101)}

lsvc = SVC(kernel='linear', max_iter=25000, random_state=0)
lsvc_gs = BayesSearchCV(lsvc,param,n_iter=niter, cv=cv, n_jobs=-1, refit=True, random_state=0)
lsvc_gs.fit(X_train_v, y_train_l);

In [ ]:
show_html(pd.DataFrame(lsvc_gs.cv_results_).loc[:,['params', 'mean_test_score','rank_test_score']].sort_values(by='rank_test_score').head().to_html())

Podemos ver que la separabilidad no es perfecta con este número de atributos, los artículos de astrofísica parecen ser los que se clasficican mejor.

In [ ]:
print(classification_report(lsvc_gs.predict(X_test_v), y_test_l,target_names=cls))
results_df = save_results(lsvc_gs, X_test_v, y_test_l, 'linear SVM binary', results_df)

Los artículos de física son los que tienen más confusión con el resto, los de astrofísica no se parecen tanto a los de CS o de matematicas y entre estos dos también hay cierto solapamiento.

In [ ]:
plt.figure(figsize=(8,8));
ConfusionMatrixDisplay.from_estimator(lsvc_gs, X_test_v, y_test_l, display_labels=cls, ax=plt.subplot());

En la curva ROC también podemos ver que los articulos de astrofísica son los que se distinguen mejor.

In [ ]:
plt.figure(figsize=(8,8));
roc_auc(lsvc_gs, X_train_v, y_train_l, X_test_v, y_test_l, classes=cls);

#### Kernels polinómicos

Ahora probaremos usando kernels no lineales empezando por kernels polinómicos cuadráticos y cúbicos

In [ ]:
param = {'C':10**np.linspace(-3,3,101), 'degree':[2,3]}

psvc =  SVC(kernel='poly', max_iter=25000, random_state=0)
psvc_gs = BayesSearchCV(psvc,param,n_iter=niter, cv=cv, n_jobs=-1, refit=True, random_state=0)
psvc_gs.fit(X_train_v, y_train_l);

Dependiendo de la exploración podemos llegar a un resultado marginalmente mejor

In [ ]:
show_html(pd.DataFrame(psvc_gs.cv_results_).loc[:,['params', 'mean_test_score','rank_test_score']].sort_values(by='rank_test_score').head().to_html())

Obtenemos un resultado ligeramente mejor en el test, hemos perdido algo en los artículos de astrofísica, pero hemos ganado algo en el resto

In [ ]:
print(classification_report(psvc_gs.predict(X_test_v), y_test_l,target_names=cls))
results_df = save_results(psvc_gs, X_test_v, y_test_l, 'polynomial SVM binary', results_df)

Podemos ver un patron de confusión similar

In [ ]:
plt.figure(figsize=(8,8));
ConfusionMatrixDisplay.from_estimator(psvc_gs, X_test_v, y_test_l, display_labels=cls, ax=plt.subplot());

La curva ROC parece bastante similar.

In [ ]:
plt.figure(figsize=(8,8));
roc_auc(psvc_gs, X_train_v, y_train_l, X_test_v, y_test_l, classes=cls);

#### Kernel RBF

Probamos ahora con el kernel gausiano

In [ ]:
param = {'C':10**np.linspace(-3,3,101), 'gamma':['scale','auto']}

rbsvc =  SVC(kernel='rbf', max_iter=25000, random_state=0)
rbsvc_gs = BayesSearchCV(rbsvc,param,n_iter=niter, cv=cv, n_jobs=-1, refit=True, random_state=0)
rbsvc_gs.fit(X_train_v, y_train_l);

Dependiendo de la exploración vemos otra ligera mejora en el resultado

In [ ]:
show_html(pd.DataFrame(rbsvc_gs.cv_results_).loc[:,['params', 'mean_test_score','rank_test_score']].sort_values(by='rank_test_score').head().to_html())

En el test se equilibran algo mas los resultados entre las clases con más confusión.

In [ ]:
print(classification_report(rbsvc_gs.predict(X_test_v), y_test_l,target_names=cls))
results_df = save_results(rbsvc_gs, X_test_v, y_test_l, 'RBF SVM binary', results_df)

Probablemente hay artículos que combinan más de un tema y eso hace difícil distinguirlos.

In [ ]:
plt.figure(figsize=(8,8));
ConfusionMatrixDisplay.from_estimator(rbsvc_gs, X_test_v, y_test_l, display_labels=cls, ax=plt.subplot());

La curva ROC es ligeramente mejor para alguna clase

In [ ]:
plt.figure(figsize=(8,8));
roc_auc(rbsvc_gs, X_train_v, y_train_l, X_test_v, y_test_l, classes=cls);

### TF-IDF como atributos

Ahora podemos usar la frecuencia de las palabras en los documentos (TF) y su fecuencia en el corpus (IDF) como atributos. Eso hará que no solo usemos las palabras que aparecen en los textos, sino que también tengamos en cuenta su importancia en el documento y en el corpus de texto.

In [ ]:
cvect = TfidfVectorizer(stop_words='english', max_features=voc_size)

X_train_vt = cvect.fit_transform(X_train)
X_test_vt = cvect.transform(X_test)
X_train_vt.shape

Podemos visualizar el vocabulario que tenemos, hay ligeras diferencias dado que estamos dando ahora pesos a las palabras.

In [ ]:
from wordcloud import WordCloud

wordcloud = WordCloud(background_color='white').generate_from_frequencies(cvect.vocabulary_)
plt.figure(figsize=(12,8))
plt.imshow(wordcloud, interpolation='bilinear');
plt.axis('off');

### Visualización

La proyección de PCA ha cambiado ligeramente, la relación espacial entre clase se ha mantenido, pero los ejemplos se han dispersado algo más con menor variancia en algunas direcciones.

In [ ]:
pca = PCA()
tdata = pca.fit_transform(X_train_vt.todense())
dfdata = pd.concat([pd.DataFrame(tdata[:,:2]),pd.DataFrame({'label':y_train})],axis=1)
fig = plt.figure(figsize=(8,8))
sns.scatterplot(x=0, y=1, hue='label', data=dfdata, palette='tab10');


Podemos usar también t-SNE inicializado con PCA. 

In [ ]:
tsne = TSNE(init='pca')
tdata = tsne.fit_transform(X_train_vt.todense())
dfdata = pd.concat([pd.DataFrame(tdata[:,:2]),pd.DataFrame({'label':y_train})],axis=1)
fig = plt.figure(figsize=(8,8))
sns.scatterplot(x=0, y=1, hue='label', data=dfdata, palette='tab10');

Podemos ver las mismas relaciones, quizás algo más de separabilidad que antes.

### Support Vector Machines - TFIDF

#### SVM lineal

Vamos a aplicar las diferentes SVMs a esta nueva representación empezando con la SVM lineal.

En este caso no tenemos datos binarios ahora si podemos normalizarlos, habitualmente eso ayuda a la convergencia.

In [ ]:
mmscaler = MinMaxScaler()

X_train_vt_m = mmscaler.fit_transform(X_train_vt.todense())
X_test_vt_m = mmscaler.transform(X_test_vt.todense())

In [ ]:
param = {'C':10**np.linspace(-3,3,101)}

lsvc =  SVC(kernel='linear', max_iter=25000, random_state=0)
lsvc_gs = BayesSearchCV(lsvc,param,n_iter=niter, cv=cv, n_jobs=-1, refit=True, random_state=0)
lsvc_gs.fit(X_train_vt_m, y_train_l);

In [ ]:
show_html(pd.DataFrame(lsvc_gs.cv_results_).loc[:,['params', 'mean_test_score','rank_test_score']].sort_values(by='rank_test_score').head().to_html())

No hay una gran diferencia con la otra represesentación

In [ ]:
print(classification_report(lsvc_gs.predict(X_test_vt_m), y_test_l,target_names=cls))
results_df = save_results(lsvc_gs, X_test_vt_m, y_test_l, 'linear SVM TFIDF', results_df)

In [ ]:
plt.figure(figsize=(8,8));
ConfusionMatrixDisplay.from_estimator(lsvc_gs, X_test_vt_m, y_test_l, display_labels=cls, ax=plt.subplot());

La confusión entre clases sigue siendo parecida y la curva ROC parece algo más suave en la clase más difícil

In [ ]:
plt.figure(figsize=(8,8));
roc_auc(lsvc_gs, X_train_vt_m, y_train_l, X_test_vt_m, y_test_l, classes=cls);

#### Kernel Polinómico

In [ ]:
param = {'C':10**np.linspace(-3,3,101), 'degree':[2,3]}

psvc =  SVC(kernel='poly', max_iter=25000, random_state=0)
psvc_gs = BayesSearchCV(psvc,param,n_iter=niter, cv=cv, n_jobs=-1, refit=True, random_state=0)
psvc_gs.fit(X_train_vt_m, y_train_l);

In [ ]:
show_html(pd.DataFrame(psvc_gs.cv_results_).loc[:,['params', 'mean_test_score','rank_test_score']].sort_values(by='rank_test_score').head().to_html())

Los resultados son algo peores, per no hay gran diferencia

In [ ]:
print(classification_report(psvc_gs.predict(X_test_vt_m), y_test_l,target_names=cls))
results_df = save_results(psvc_gs, X_test_vt_m, y_test_l, 'polynomial SVM TFIDF', results_df)

In [ ]:
plt.figure(figsize=(8,8));
ConfusionMatrixDisplay.from_estimator(psvc_gs, X_test_vt_m, y_test_l, display_labels=cls, ax=plt.subplot());

Podemos ver que la curva ROC de la clase más difícil se ha acercado a la de las clases CS y math, se ha equilibrado más la relacón entre precisión y recuperación.

In [ ]:
plt.figure(figsize=(8,8));
roc_auc(psvc_gs, X_train_vt_m, y_train_l, X_test_vt_m, y_test_l, classes=cls);

### Kernel RBF

In [ ]:
param = {'C':10**np.linspace(-3,3,101), 'gamma':['scale','auto']}

rbsvc =  SVC(kernel='rbf', max_iter=25000, random_state=0)
rbsvc_gs = BayesSearchCV(rbsvc,param,n_iter=niter, cv=cv, n_jobs=-1, refit=True, random_state=0)
rbsvc_gs.fit(X_train_vt_m, y_train_l);

In [ ]:
show_html(pd.DataFrame(rbsvc_gs.cv_results_).loc[:,['params', 'mean_test_score','rank_test_score']].sort_values(by='rank_test_score').head().to_html())

Tenemos resultados también parecidos a los de antes

In [ ]:
print(classification_report(rbsvc_gs.predict(X_test_vt_m), y_test_l,target_names=cls))
results_df = save_results(rbsvc_gs, X_test_vt_m, y_test_l, 'RBF SVM TFIDF', results_df)

In [ ]:
plt.figure(figsize=(8,8));
ConfusionMatrixDisplay.from_estimator(rbsvc_gs, X_test_vt_m, y_test_l, display_labels=cls, ax=plt.subplot());

In [ ]:
plt.figure(figsize=(8,8));
roc_auc(rbsvc_gs, X_train_vt_m, y_train_l, X_test_vt_m, y_test_l, classes=cls);

Estos son los resultados finales, si elegimos el mejor modelos según el acierto en el conjunto de test entonces 
nos decidiriamos por la SVM con kernel RBF que da un resultado ligeramente mejor. Podemos ver que también es mejor en el rsto de medidas.

In [ ]:
results_df.sort_values(by=['test acc'], ascending=False)

#### Interpretación de resultados

Es complicado el poder interpretar directamente este modelo a partir de los pesos incluso con los pesos lineales, podríamos hacernos una idea viendo los pesos que corresponden a cada separador, pero hay demasiados atributos.

Una cosa que podemos hacer es interpretar los resultados de la clasificación de ejemplos.

En teoría explicamos LIME que permite dar una interpretación de las clasificaciones de un ejemplo generando ejemplos alrededor del que queremos explicar obteniendo la respuesta del clasificador y aprendiendo un clasificador surrogado lineal que permite interpretar los pesos de ese clasificador.

En la librería ELI5 tenemos implementado LIME y además un método que nos permite hacer una interpretación visual para texto.

En este caso se generan ejemplos a partir de ir eliminando aleatoriamente un número de palabras del ejemplo que queremos interpretar, obteniendo las respuestas del clasificador y entrenando una regresión logística de la que sacaremos los pesos y obtendremos cuales son los pesos que determinan la clasificación en una clase u otra.

Para poder hacer esto necesitamos que el clasificador obtenga probabilidades, asi que reajustaremos el mejor modelo de esta manera.

Crearemos un modelo que reciba un texto, lo transforme a un vector binario y luego lo clasifique con la SVM con kernel RBF

In [ ]:
from sklearn.pipeline import  make_pipeline

clf = SVC(C= 1, kernel='rbf', gamma='scale', random_state=0, probability=True);

pipe = make_pipeline(cvec, clf);

pipe.fit(X_train, y_train_l);
pipe.score(X_test, y_test_l);

podemos elegir un ejemplo a explicar

In [ ]:
s=522
X_test[s], y_test[s]

Entrenamos el modelo para explicar texto de ELI5

In [ ]:
import eli5
from eli5.lime import TextExplainer
from eli5.lime.samplers import MaskingTextSampler

te = TextExplainer(sampler= MaskingTextSampler(), random_state=42);
te.fit(X_test[s], pipe.predict_proba);

Obtenemos la explicación para cada clase. El color verde con su intensidad nos indica que palabras son positivas para clasificarlo en esa clase los rojos lo que son negativos para su clasificación en esa clase. 

In [ ]:
te.show_prediction(target_names=list(cls))

## Sección 2: Energy Data - Regresión

Como vimos en la última sesión de laboratorio, un problema que se puede resolver mediante regresión es la predicción de series de tiempo.

En este tipo de problemas queremos predecir valores de un momento en el tiempo a partir de los valores anteriores. En este caso debemos decidir cuantos instantes anteriores utilizamos y si utilizamos solo la variable objetivo o añadimos también otras variables que tengamos disponibles.

Si recordais, en este problema queremos predecir el consumo de energia de los electrodomesticos de una casa. Tenemos muchos otros atributos, pero en este caso solo utilizaremos la variable objetivo. Podéis encontrar la documentación de este conjunto de datos aqui (https://archive.ics.uci.edu/ml/datasets/Appliances+energy+prediction)

Podemos usar la SVM para regresión para realizar la misma tarea que en el notebook de Knn y MLP.

In [ ]:
from numpy.lib.stride_tricks import sliding_window_view
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, make_scorer
pd.set_option('display.precision', 5)

niter = 15
cv = 5

In [ ]:
data = load_energy()

In [ ]:
energy = data.loc[:,'Appliances']

Realizamos el mismo preproceso para poder comparar los resultados, de todas formas diferentes preprocesos pueden tener un impacto en la calidad del resultado. Una posibilidad es usar la serie a predecir sin preprocesar, es algo que podeis hacer vosotros.

In [ ]:
e_train, e_test =  energy.iloc[:12000], energy.iloc[12000:]

w = 4

sdscaler = MinMaxScaler()

e_train_s = sdscaler.fit_transform(e_train.to_numpy().reshape(-1, 1))
e_test_s = sdscaler.transform(e_test.to_numpy().reshape(-1, 1))

windows_train = sliding_window_view(e_train_s, w+1, axis=0).copy()
X_train_w, y_train_w = windows_train.squeeze()[:,:-1], windows_train.squeeze()[:,-1]

windows_test = sliding_window_view(e_test_s, w+1, axis=0).copy()
X_test_w, y_test_w = windows_test.squeeze()[:,:-1], windows_test.squeeze()[:,-1]

#### Kernel lineal

La SVM para regresión utiliza la epsilon insensitive loss. El valor del parámetro `epsilon` depende de la escala de los datos, en este caso [0,1], dependiendo del valor que le demos en este caso es posible que eso nos de un valor mínimo que pueda generar la regresión, asi que hemos de tener cuidado al usarlo y ver el efecto que tiene en la predicción.

In [ ]:
param = {'C':10**np.linspace(-3,3,101), 'epsilon':np.linspace(0,0.01,11)}

lsvr =  SVR(kernel='linear', max_iter=25000, cache_size=2000)
lsvr_gs = BayesSearchCV(lsvr,param,n_iter=niter, 
                        cv=TimeSeriesSplit(n_splits=cv, gap=w+1), 
                        scoring=make_scorer(mean_squared_error, greater_is_better=False),
                        n_jobs=-1, 
                        refit=True, random_state=0)
lsvr_gs.fit(X_train_w, y_train_w);

In [ ]:
show_html(pd.DataFrame(lsvr_gs.cv_results_).loc[:,['params', 'mean_test_score','rank_test_score']].sort_values(by='rank_test_score').head().to_html())


Podemos ver que tenemos una predicción parecida a la de los otros dos modelos, ligeramente peor.

In [ ]:
mean_squared_error(y_test_w,lsvr_gs.predict(X_test_w)), mean_absolute_error(y_test_w,lsvr_gs.predict(X_test_w))

In [ ]:
plt.figure(figsize=(18,10))
plt.plot(y_test_w[:500],'r');
plt.plot(lsvr_gs.predict(X_test_w[:500,:]),'b');

Podemos ver también que la predicción en el conjunto de test también va siguiendo los valores reales.

En los valores más bajos de la serie podemos ver que muchas veces la predicción es casi constante, eso puede tener que ver con el valor de `epsilon` que estamos usando. Eso no quiere decir que sea un problema, los valores mas bajos podrían ser básicamente ruido así que una predicción más suave podría ser más plausible, pero depende de la aplicación.

### Kernel RBF

Probamos ahora con el kernel RBF.

In [ ]:
param = {'C':10**np.linspace(-3,3,101), 'gamma':['scale','auto'], 'epsilon':np.linspace(0,0.01,11)}

rbsvr =  SVR(kernel='rbf', max_iter=25000, cache_size=2000)
rbsvr_gs = BayesSearchCV(rbsvr,param,n_iter=niter, 
                        cv=TimeSeriesSplit(n_splits=cv, gap=w+1), 
                        scoring=make_scorer(mean_squared_error, greater_is_better=False),
                        n_jobs=-1, 
                        refit=True, random_state=0)
rbsvr_gs.fit(X_train_w, y_train_w);

In [ ]:
show_html(pd.DataFrame(rbsvr_gs.cv_results_).loc[:,['params', 'mean_test_score','rank_test_score']].sort_values(by='rank_test_score').head().to_html())


El resultado es ligeramente mejor, más cerca al de los otros dos modelos.

In [ ]:
mean_squared_error(y_test_w,rbsvr_gs.predict(X_test_w)), mean_absolute_error(y_test_w,rbsvr_gs.predict(X_test_w))

In [ ]:
plt.figure(figsize=(18,10))
plt.plot(y_test_w[:500],'r');
plt.plot(rbsvr_gs.predict(X_test_w[:500,:]),'b');

También sigue bastante bien los datos de test.

In [ ]:

print(f"Total Running time {timedelta(seconds=(time() - init_time))}")